# Download e Preparação de Datasets Médicos

Este notebook é responsável por baixar e preparar os datasets necessários para o pipeline:

- **ACDC (Automated Cardiac Diagnosis Challenge):** Dataset de segmentação cardíaca
- **BraTS (Brain Tumor Segmentation):** Dataset de segmentação de tumores cerebrais
- **BTCV (Beyond the Cranial Vault):** Dataset de segmentação de múltiplos órgãos (alternativa)

## Estrutura dos Datasets

Após o download, os datasets serão organizados da seguinte forma:
```
data/
├── ACDC/
│   ├── training/
│   └── testing/
├── BraTS/
│   ├── training/
│   └── validation/
└── processed/
    ├── ACDC/
    └── BraTS/
```


In [ ]:
import os
import requests
import zipfile
import tarfile
from pathlib import Path
import shutil

# ============================================
# DETECÇÃO DE AMBIENTE (COLAB OU LOCAL)
# ============================================
try:
    import google.colab
    IN_COLAB = True
    from google.colab import drive
    drive.mount('/content/drive')
    print("✅ Google Colab detectado - Drive montado")
except ImportError:
    IN_COLAB = False
    print("✅ Ambiente local detectado")

# Configurar caminhos baseado no ambiente
if IN_COLAB:
    BASE_DIR = Path("/content/drive/MyDrive/Mestrado_TCC/datasets")
    FRAMEWORK_DIR = Path("/content/drive/MyDrive/Mestrado_TCC/Framework")
    if FRAMEWORK_DIR.exists():
        os.chdir(FRAMEWORK_DIR)
else:
    BASE_DIR = Path("../data")

# Criar diretórios
BASE_DIR.mkdir(parents=True, exist_ok=True)

ACDC_DIR = BASE_DIR / "ACDC"
BraTS_DIR = BASE_DIR / "BraTS"
PROCESSED_DIR = BASE_DIR / "processed"

# Criar diretórios
ACDC_DIR.mkdir(exist_ok=True)
BraTS_DIR.mkdir(exist_ok=True)
PROCESSED_DIR.mkdir(exist_ok=True)

print("\n📁 Estrutura de diretórios criada!")
print(f"   Diretório base: {BASE_DIR}")
print(f"   Ambiente: {'Colab' if IN_COLAB else 'Local'}")


## Download do Dataset ACDC

O dataset ACDC está disponível através do site oficial. Você precisa:
1. Criar uma conta no site: https://www.creatis.insa-lyon.fr/Challenge/acdc/
2. Baixar o dataset manualmente
3. Colocar o arquivo no diretório apropriado

**Nota:** Este notebook assume que você já baixou o dataset. Ajuste os caminhos conforme necessário.


In [ ]:
def extract_acdc_dataset(archive_path, output_dir):
    """
    Extrai o dataset ACDC do arquivo compactado.
    
    Args:
        archive_path: Caminho para o arquivo .zip ou .tar.gz do ACDC
        output_dir: Diretório de saída
    """
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    
    if archive_path.endswith('.zip'):
        with zipfile.ZipFile(archive_path, 'r') as zip_ref:
            zip_ref.extractall(output_dir)
    elif archive_path.endswith('.tar.gz') or archive_path.endswith('.tgz'):
        with tarfile.open(archive_path, 'r:gz') as tar_ref:
            tar_ref.extractall(output_dir)
    else:
        raise ValueError(f"Formato de arquivo não suportado: {archive_path}")
    
    print(f"Dataset ACDC extraído para: {output_dir}")

# Exemplo de uso (descomente e ajuste o caminho):
# ACDC_ARCHIVE = "/caminho/para/acdc_dataset.zip"  # Ajuste este caminho
# if os.path.exists(ACDC_ARCHIVE):
#     extract_acdc_dataset(ACDC_ARCHIVE, ACDC_DIR)
# else:
#     print("Arquivo ACDC não encontrado. Por favor, baixe manualmente do site oficial.")


## Download do Dataset BraTS

O dataset BraTS está disponível através do site oficial. Você precisa:
1. Criar uma conta no site: https://www.med.upenn.edu/cbica/brats2021/
2. Baixar o dataset manualmente
3. Colocar o arquivo no diretório apropriado

**Nota:** Este notebook assume que você já baixou o dataset. Ajuste os caminhos conforme necessário.


In [ ]:
def extract_brats_dataset(archive_path, output_dir):
    """
    Extrai o dataset BraTS do arquivo compactado.
    
    Args:
        archive_path: Caminho para o arquivo .zip ou .tar.gz do BraTS
        output_dir: Diretório de saída
    """
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    
    if archive_path.endswith('.zip'):
        with zipfile.ZipFile(archive_path, 'r') as zip_ref:
            zip_ref.extractall(output_dir)
    elif archive_path.endswith('.tar.gz') or archive_path.endswith('.tgz'):
        with tarfile.open(archive_path, 'r:gz') as tar_ref:
            tar_ref.extractall(output_dir)
    else:
        raise ValueError(f"Formato de arquivo não suportado: {archive_path}")
    
    print(f"Dataset BraTS extraído para: {output_dir}")

# Exemplo de uso (descomente e ajuste o caminho):
# BraTS_ARCHIVE = "/caminho/para/brats_dataset.zip"  # Ajuste este caminho
# if os.path.exists(BraTS_ARCHIVE):
#     extract_brats_dataset(BraTS_ARCHIVE, BraTS_DIR)
# else:
#     print("Arquivo BraTS não encontrado. Por favor, baixe manualmente do site oficial.")


## Preparação e Organização dos Dados

Após extrair os datasets, organizamos os dados em uma estrutura padronizada para facilitar o processamento posterior.


In [ ]:
def organize_acdc_data(source_dir, output_dir):
    """
    Organiza os dados do ACDC em uma estrutura padronizada.
    
    Estrutura esperada:
    - patient001/
      - patient001_4d.nii.gz
      - patient001_frame01.nii.gz
      - patient001_frame01_gt.nii.gz
      - ...
    """
    source_dir = Path(source_dir)
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # Listar todos os pacientes
    patients = [d for d in source_dir.iterdir() if d.is_dir()]
    
    print(f"Encontrados {len(patients)} pacientes no ACDC")
    
    # Copiar estrutura (ou criar links simbólicos para economizar espaço)
    for patient_dir in patients:
        dest_dir = output_dir / patient_dir.name
        if not dest_dir.exists():
            shutil.copytree(patient_dir, dest_dir)
    
    print(f"Dados organizados em: {output_dir}")

def organize_brats_data(source_dir, output_dir):
    """
    Organiza os dados do BraTS em uma estrutura padronizada.
    
    Estrutura esperada:
    - BraTS-Training_001/
      - BraTS-Training_001_t1.nii.gz
      - BraTS-Training_001_t1ce.nii.gz
      - BraTS-Training_001_t2.nii.gz
      - BraTS-Training_001_flair.nii.gz
      - BraTS-Training_001_seg.nii.gz
    """
    source_dir = Path(source_dir)
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # Listar todos os casos
    cases = [d for d in source_dir.iterdir() if d.is_dir()]
    
    print(f"Encontrados {len(cases)} casos no BraTS")
    
    # Copiar estrutura
    for case_dir in cases:
        dest_dir = output_dir / case_dir.name
        if not dest_dir.exists():
            shutil.copytree(case_dir, dest_dir)
    
    print(f"Dados organizados em: {output_dir}")

# Exemplo de uso:
# organize_acdc_data(ACDC_DIR / "training", PROCESSED_DIR / "ACDC" / "training")
# organize_brats_data(BraTS_DIR / "training", PROCESSED_DIR / "BraTS" / "training")
